<a href="https://colab.research.google.com/github/AKDGrant/research-agent/blob/main/ResearchAgentUp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate langchain langchain_community ddgs --quiet

In [2]:
from google.colab import userdata
from huggingface_hub import login

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from ddgs import DDGS

In [3]:
HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN)

In [ ]:
model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0  # Use GPU if available
)

llm = HuggingFacePipeline(pipeline=pipe)

In [5]:
prompt = PromptTemplate(
    input_variables=["query"],
    template="Summarize the following information clearly and concisely: {query}"
)

def search_web(query):
    """Search the web using DuckDuckGo and return top 3 results."""
    results = DDGS().text(query)
    return [r['body'] for r in results[:3]]

In [8]:
query = "latest AI trends 2025"

web_results = search_web(query)

if not web_results:
    web_results = ["No results found. Using placeholder text for demo purposes."]

full_text = " ".join(web_results)

chain = LLMChain(llm=llm, prompt=prompt)
response = chain.run({"query": full_text})

print("=== Web Results ===\n")
for i, r in enumerate(web_results, 1):
    print(f"Result {i}:\n{r}\n{'-'*50}\n")
print("=== Summary ===\n")

for line in response.split('. '):
    print(f"- {line.strip()}")

=== Web Results ===

Result 1:
View the latest news and breaking news today for U.S., world, weather, entertainment, politics and health at CNN.com.
--------------------------------------------------

Result 2:
Read the latest headlines, breaking news, and videos at APNews.com, the definitive source for independent journalism from every corner of the globe.
--------------------------------------------------

Result 3:
Visit BBC News for the latest news, breaking news, video, audio and analysis. BBC News provides trusted World, U.S. and U.K. news as well as local and regional perspectives.
--------------------------------------------------

=== Summary ===

- Visit CNN.com
- Visit APNews.com
- Visit BBC News.
